# Mode Selection
Set finetuning = 1 if you want to finetune, or set finetuning = 0 if you want to just load the saved model to get inference.

In [1]:
finetuning = 0
inference = not finetuning

# Drive Mount

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# Read Data

In [2]:
import pandas as pd
data = pd.read_csv('/kaggle/input/gb-all-necessary-updated/train_test_hatespeech_data.csv')
data

,male,female,_original_label,test
0,যত্তসব পাপন শালার ফাজলামী!!!!!,যত্তসব জাহান শালীর ফাজলামী!!!!!,1,0
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,নাহার শালী রে রিমান্ডে নেওয়া দরকার,1,0
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,নূর জাহান ম্যাডামের মেয়ে এতো বড় জারজ হবে এটা ...,1,0
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,শালী লুচ্চা দেখতে পাঠীর মত দেখা যায়,1,1
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,তুই তো শালী গাজা খাইছচ । তুর মার হেডায় খেলবে স...,1,0
...,...,...,...,...
9997,ভাই এরকম ভিডিও আরো করেন,আফা এরকম ভিডিও আরো করেন,0,0
9998,ভাই এরকম ভিডিও আরো করেন,বোন এরকম ভিডিও আরো করেন,0,0
9999,ভাই এরকম ভিডিও আরো করেন,ভাবী এরকম ভিডিও আরো করেন,0,0
10000,মূর্খ খালেদার ফাঁসি চাই,মূর্খা দুর্গা ফাঁসি চাই,0,0


# Train, Test, Validation Split
We are going to consider the data as test data if the prediction between original sentence and NER converted sentence become unequal.

In [3]:
temp = data[data['test']!= 1]
test = data[data['test']== 1]

In [4]:
len(temp), len(test)

(8002, 2000)

In [5]:
t1 = temp[['male', '_original_label']]
t2 = temp[['female', '_original_label']]
t2.columns = ['male', '_original_label']
temp = pd.concat([t1, t2], axis = 0)
temp

,male,_original_label
0,যত্তসব পাপন শালার ফাজলামী!!!!!,1
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,1
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,1
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,1
5,তোর কপালে জুতা মারি শালার পুত,1
...,...,...
9997,আফা এরকম ভিডিও আরো করেন,0
9998,বোন এরকম ভিডিও আরো করেন,0
9999,ভাবী এরকম ভিডিও আরো করেন,0
10000,মূর্খা দুর্গা ফাঁসি চাই,0


In [6]:
validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=101))
train = temp.drop(validation.index)

<ipython-input-6-cf4ecadbd9c2>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=101))


In [7]:
train

,male,_original_label
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,1
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,1
6,পাপনে পাগল হয়াছে,1
11,শালায় আমার একটা পছন্দের প্লেয়ার ছিল কিন্তু র...,1
12,পাপনের পদত্যক চাই শালায় চোর,1
...,...,...
9997,আফা এরকম ভিডিও আরো করেন,0
9998,বোন এরকম ভিডিও আরো করেন,0
9999,ভাবী এরকম ভিডিও আরো করেন,0
10000,মূর্খা দুর্গা ফাঁসি চাই,0


In [8]:
validation

,male,_original_label
9409,জাযাকাল্লাহ খায়ের চালিয়ে যান ভাই জয় হবে একদ...,0
9545,ভাই সব মন্এী চোর এটা তো জানা কথা।,0
7049,তর সমান স্ত্রীলোক না দেকলে কে দেখবে তর ভোন,0
5963,আরে ভাই গালি দিয়েছে তা এতো কি হয়েছে ।,0
8504,আমার খারাপ লাগেনি অমানুষের কান্না দেখে একটা মে...,0
...,...,...
4232,কনডম ফেটে যাদের জন্ম হয়েছে সেসব কোটার গাই গুলো...,1
3449,কিছু বলার ভাষা হারিয়ে ফেলেছি ভাবী এই গুলো কি শ...,1
2194,এখানে ভাই বিরাট ঝামেলা আছে কোপানোর কিছুক্ষণ পর...,1
2781,তরে থাপ্পড় দিয়ে টেস থেকে নামা হারামজাদী,1


In [9]:
test

,male,female,_original_label,test
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,শালী লুচ্চা দেখতে পাঠীর মত দেখা যায়,1,1
8,দেখেছো মাইগ্যা না হিজরা বোঝা যাচ্ছে না । দেখে...,দেখেছো মাগি না হিজরা বোঝা যাচ্ছে না। দেখে মনে ...,1,1
10,সব মানুষ আপন হয় না জেমন পাপন আপন হইতে পারে না ...,সব মানুষ আপন হয় না জেমন রাইসা আপন হইতে পারে না...,1,1
17,এই চুদির ভাই তুই কোথায় শুনলি সাকিব ইংল্যান্ডেে...,এই চুদির আফা তুই কোথায় শুনলি সাকিব ইংল্যান্ডেে...,1,1
20,পাপন সালা পাগল হয়ে গেছে সবাই মিলে একে জুতা মারো,সাহানা সালী পাগলী হয়ে গেছে সবাই মিলে একে জুতা...,1,1
...,...,...,...,...
9961,স্বামী স্ত্রী কে ভালবেসে মিষ্টি খাওয়াইছে বলেএই...,স্ত্রী স্বামী কে ভালবেসে মিষ্টি খাওয়াইছে বলেএই...,0,1
9966,রানু পাগলি কে খমা করা উচিত নয়,আহনাফ পাগলা কে খমা করা উচিত নয়,0,1
9969,তাহসানের উচিৎ ভালো ছেলে দেখে বিয়ে করা বাজে একট...,তাহসানের উচিৎ ভালো মেয়ে দেখে বিয়ে করা বাজে একট...,0,1
9987,শাবনুর মহা নায়ক,শাবনুর মহা নায়িকা,0,1


# Load Pretrained Model

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transform
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig

def load_transformer_based_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForMaskedLM.from_pretrained(model_path ,output_hidden_states=True)
    return tokenizer, model
model_path = "Hate-speech-CNERG/bengali-abusive-MuRIL"
tokenizer, b_model = load_transformer_based_model(model_path)
print('Total Pretrained tokens: ',len(tokenizer.get_vocab()))
tokenizer.add_tokens(['<Name>' ,'<Gender>'])
b_model.resize_token_embeddings(len(tokenizer))
print('Total number of tokens after adding the new ones: ',len(tokenizer.get_vocab()))

tokenizer_config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/950M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:818: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output

Total Pretrained tokens:  197285


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Total number of tokens after adding the new ones:  197287


# Dataset Preparation

In [15]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm



class BertDataset(Dataset):
    def __init__(self, data, tokenizer, train , gender, max_length):
        super(BertDataset, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length
        self.train = train
        self.gender = gender

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        if self.train == 1:
            text1 = self.train_csv.iloc[index,0] #0th column all data
        else:
            if self.gender == 'male':
                text1 = self.train_csv.iloc[index,0] #male
            else:
                text1 = self.train_csv.iloc[index,1] #female
                
                

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        
        try:
            if self.train == 1:
              return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 1], dtype=torch.long)
            else:
              return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 2], dtype=torch.long)
                
        except:
          print(index, self.train_csv[index])

dataset_train= BertDataset(train, tokenizer, train = 1 , gender = None, max_length = 512)
dataset_val= BertDataset( validation, tokenizer, train = 1 , gender = None, max_length = 512)
dataset_test_male = BertDataset(test, tokenizer, train = 0 , gender = 'male', max_length = 512)
dataset_test_female = BertDataset(test, tokenizer, train = 0 , gender = 'female', max_length = 512)

train_loader=DataLoader(dataset=dataset_train,batch_size=8, shuffle=True)
validation_loader=DataLoader(dataset=dataset_val,batch_size=8, shuffle=False)
test_loader_male=DataLoader(dataset=dataset_test_male,batch_size=8, shuffle=False)
test_loader_female=DataLoader(dataset=dataset_test_female,batch_size=8, shuffle=False)

In [13]:
# id,am, tid, l = next(iter(train_loader))
# x = b_model(id, am,tid)
# x.hidden_states[0].shape

# Model

In [16]:

class BERT(nn.Module):
    def __init__(self, model):
        super(BERT, self).__init__()
        self.bert_model = model
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(768, 1)

    def forward(self,ids,mask,token_type_ids):
        o2= self.bert_model(ids,mask,token_type_ids).hidden_states[0]
        o2= o2.max(dim=1).values
        o2 = self.dropout(o2)
        # print('------->>>>>>>>>',o2.values.shape)
        out= self.out(o2)

        return out, o2

masked_model=BERT(b_model)
# loss_fn = nn.BCEWithLogitsLoss()
# #Initialize Optimizer
# optimizer= optim.Adam(model.parameters(),lr= 0.0001)

In [15]:
# id,am, tid, l = next(iter(train_loader))
# x,_ = masked_model(id, am,tid)
# x.shape

# Training Loop Decorator

In [17]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product
import json
from torch.utils.tensorboard import SummaryWriter
import time
import torchvision
import pandas as pd
import torch
from IPython.display import display
from IPython.display import clear_output

class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple('Run', params.keys())


        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))


        return runs


class RunManager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None

        self.epoch_valid_loss = 0.0
        self.epoch_num_valid_correct = 0

        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None

        self.network = None
        self.loader = None
        self.validation_loader = None
        self.tb = None

    def begin_run(self, run, network, loader, validation_loader):
        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1

        self.network = network
        self.loader = loader
        self.validation_loader = validation_loader
        self.tb = SummaryWriter(comment=f'-{run}')



    def end_run(self):
        self.tb.close()
        self.epoch_count = 0

    def begin_epoch(self):
        self.epoch_start_time = time.time()
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_valid_loss = 0
        self.epoch_num_valid_correct = 0

    def end_epoch(self):
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        loss = self.epoch_loss/len(self.loader.dataset)
        accuracy = self.epoch_num_correct/len(self.loader.dataset)
        val_loss = self.epoch_valid_loss/len(self.validation_loader.dataset)
        val_accuracy = self.epoch_num_valid_correct/len(self.validation_loader.dataset)

        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        self.tb.add_scalar('Validation Loss', val_loss, self.epoch_count)
        self.tb.add_scalar('Validation Accuracy', val_accuracy, self.epoch_count)

        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)

        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["validation loss"] = val_loss
        results["validation accuracy"] = val_accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')

        clear_output(wait=True)
        display(df)

    def track_loss(self, loss):
        self.epoch_loss += loss.item()#*self.loader.batch_size

    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)

    def track_validation_loss(self, loss):
        self.epoch_valid_loss += loss.item()#*self.loader.batch_size

    def track_num_validation_correct(self, preds, labels):
        self.epoch_num_valid_correct += self._get_num_correct(preds, labels)

    @torch.no_grad()
    def _get_num_correct(self, preds, labels):
        return torch.where(preds >= 0, 1, 0).eq(labels).sum().item()

    def save(self, fileName):
        pd.DataFrame.from_dict(
            self.run_data,
            orient='columns').to_csv(rf'{fileName}.csv')


# Training

In [18]:
SAVE_MODEL = True
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup, AdamW
def my_trainer(train_loader, validation_loader, network, LR = 1e-5 ,epochs = 5, BATCH_SIZE = 16,  fresh_training = True, validation = True):
    params = OrderedDict(
        lr = [LR],
        batch_size =[BATCH_SIZE],
        device = ['cuda' if torch.cuda.is_available() else 'cpu'],
    )

    m = RunManager()
    for run in RunBuilder.get_runs(params):
        device = torch.device(run.device)
        network = network.to(device)
        # optimizer = optim.Adam(network.parameters(), lr=run.lr)

        loss_fn = nn.BCEWithLogitsLoss()
        total_steps = len(train_loader) * epochs
        optimizer = optim.Adam(network.parameters(), lr= run.lr)

        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)
        if not fresh_training:
            load_model(network, optimizer, PATH = 'foa_hatespeech.pth')
        m.begin_run(run, network, train_loader, validation_loader)
        for epoch in range(epochs):
            network.train(True)
            print(f'Epoch {epoch + 1} : ', end = '\t')
            m.begin_epoch()
            for batch in tqdm(train_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)

                preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                labels = labels.type_as(preds).view(-1, 1)
                # print(labels.shape)
                # print('----------')
                # print(preds.shape)


                loss = loss_fn(preds, labels)
                # print(preds.shape, labels.shape)
                # print(preds, labels)


                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(network.parameters(), 1.0)
                optimizer.step()
                scheduler.step()


                m.track_loss(loss)
                m.track_num_correct(preds, labels)


            network.eval()
            for batch in tqdm(validation_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)
                with torch.no_grad():
                    preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                    labels = labels.type_as(preds).view(-1, 1)
                    loss = loss_fn(preds, labels)
                    m.track_validation_loss(loss)
                    m.track_num_validation_correct(preds, labels)
                # break #not doing validation here




            m.end_epoch()
            network.train(False)
        m.end_run()
    if SAVE_MODEL == True:
        save_model(network, optimizer)

    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'results_{s}'
    m.save(PATH)
    return


def save_model(network, optimizer):
    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'foa_hatespeech.pth'
    torch.save({
    'model_state_dict': network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    }, PATH)
    print(f'Saved model and optimizer at {PATH}')
    return

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH)
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return

def get_parameter_info(network, details = False):
    print('Total parameters:' ,(sum(p.numel() for p in network.parameters())))
    print('Total trainable parameters:' ,(sum(p.numel() for p in network.parameters() if p.requires_grad)))
    if details:
        print('---------------------------------\nDetailed Parameter Info: ')

        for name, parameter in network.named_parameters():
            print('name, parameter.requires_grad, parameter.is_cuda, parameter.size(): ', name, parameter.requires_grad, parameter.is_cuda, parameter.size())
            print('---------------------------------')
    return


if finetuning:
  my_trainer(train_loader, validation_loader, masked_model, 0.0001, epochs = 15, BATCH_SIZE = 16, fresh_training = True)

,run,epoch,loss,accuracy,validation loss,validation accuracy,epoch duration,run duration,lr,batch_size,device
0,1,1,0.054726,0.810406,0.037069,0.883437,564.260063,564.262074,0.0001,16,cuda
1,1,2,0.025318,0.924456,0.030834,0.902500,565.085827,1134.854559,0.0001,16,cuda
2,1,3,0.015614,0.956202,0.030443,0.910000,565.170094,1705.512941,0.0001,16,cuda
3,1,4,0.009998,0.975113,0.029179,0.918750,565.230597,2276.255640,0.0001,16,cuda
4,1,5,0.006478,0.984519,0.028381,0.919687,565.279204,2847.149708,0.0001,16,cuda
5,1,6,0.004412,0.990202,0.029934,0.918750,565.135595,3417.626776,0.0001,16,cuda
6,1,7,0.003069,0.993533,0.030233,0.920937,565.376585,3988.294793,0.0001,16,cuda
7,1,8,0.002185,0.995885,0.031411,0.918750,565.458199,4559.241636,0.0001,16,cuda
8,1,9,0.001647,0.996669,0.032342,0.919375,565.444645,5130.166335,0.0001,16,cuda
9,1,10,0.001401,0.997354,0.032173,0.920000,565.330112,5700.605597,0.0001,16,cuda


Saved model and optimizer at foa_hatespeech.pth


# Load Model

In [17]:
#Load Model
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup, AdamW

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return


if inference:
  load_model(masked_model, None, PATH = '/kaggle/input/hatespeech_foa/pytorch/default/1/foa_hatespeech.pth')

<ipython-input-17-cca5971061c9>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(PATH, map_location=torch.device('cpu'))


Loaded model from /content/drive/MyDrive/#Research/# GB/saved_model/baseline_senti.pth


# Inference

In [20]:
def get_prediction(data_loader, network):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  device = torch.device(device)
  network = network.to(device)
  network.eval()
  l = []
  emb = []
  for batch in tqdm(data_loader):
    input_ids, token_type_ids, attention_masks, labels = batch
    input_ids = input_ids.to(device)
    token_type_ids = token_type_ids.to(device)
    attention_masks = attention_masks.to(device)
    labels = labels.type(torch.LongTensor)
    labels = labels.to(device)
    with torch.no_grad():
        preds, hidden_states = network(input_ids, attention_masks,token_type_ids)
        labels = labels.type_as(preds).view(-1, 1)
        preds =  torch.where(preds >= 0, 1, 0)
        l.append(preds)
        emb.append(hidden_states)
  return l, torch.vstack(emb).cpu().numpy()

import warnings 
warnings.filterwarnings('ignore') 

p, emb_male = get_prediction(test_loader_male, masked_model)
test['pred_male_fod'] = [x[0] for pred in p for x in pred.tolist()]
p, emb_male = get_prediction(test_loader_female, masked_model)
test['pred_female_fod'] = [x[0] for pred in p for x in pred.tolist()]
x = sum(test['pred_male_fod']!=test['_original_label'])
print('For Male:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test))) 
x = sum(test['pred_female_fod']!=test['_original_label'])
print('For Female:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test)))
print('Male Female Mismatch: ', sum(test['pred_male_fod']!=test['pred_female_fod']))

100%|██████████| 250/250 [01:15<00:00,  3.29it/s]


For Male:
Total test data:  2000 Total mismatch:  255 Accuracy:  0.8725
For Female:
Total test data:  2000 Total mismatch:  251 Accuracy:  0.8745
Male Female Mismatch:  74


# FOA Result

In [ ]:
import numpy as np
np.random.seed(0)

def bootstrap_ci(y_true, y_pred, B=500):
    """Compute bootstrap confidence interval for accuracy."""
    N = len(y_true)
    original_acc = np.mean(y_pred == y_true)

    bootstrap_accs = []
    
    for _ in range(B):
        indices = np.random.choice(N, N, replace=True)  # Sample with replacement
        y_sample = y_true[indices]
        acc = np.mean(y_pred[indices] == y_sample)
        bootstrap_accs.append(acc)
    # Compute 95% confidence interval
    lower, upper = np.percentile(bootstrap_accs, [2.5, 97.5])
    
    return original_acc, (lower, upper), np.mean(bootstrap_accs)

y_true = test['_original_label'].values  # Ground truth labels
y_pred = test['pred_male_fod'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('Male: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")

y_true = test['_original_label'].values
y_pred = test['pred_female_fod'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('\nFemale: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")
test.to_csv('2,3.revision_foa_hatespeech_result.csv', index = False)

Male: 
Accuracy: 0.8725
95% Confidence Interval: (0.8565, 0.8875)
mean accuracy: 0.871744

Female: 
Accuracy: 0.8745
95% Confidence Interval: (0.8607375, 0.8885)
mean accuracy: 0.8747849999999999


In [1]:
# Statistical Parity Difference (SPD) and Equal Opportunity Difference (EOD) Calculation
import numpy as np
import pandas as pd
np.random.seed(0)

test = pd.read_csv('revision_foa_hatespeech_result.csv')


# Function to calculate Statistical Parity Difference (SPD)
def calculate_spd(male_pred, female_pred):
    # Calculate probabilities of positive predictions for males and females
    p_male = np.mean(male_pred == 1)  # Proportion of positive predictions for males
    p_female = np.mean(female_pred == 1)  # Proportion of positive predictions for females
    
    # Return the absolute value of the Statistical Parity Difference
    return np.abs(p_male - p_female)

# Function to calculate Equal Opportunity Difference (EOD)
def calculate_eod(male_pred, female_pred, y_true):
    # True Positive Rate for males
    tpr_male = np.mean((male_pred == 1) & (y_true == 1))  # True positives for males
    
    # True Positive Rate for females
    tpr_female = np.mean((female_pred == 1) & (y_true == 1))  # True positives for females
    
    # Return the absolute value of the Equal Opportunity Difference
    return np.abs(tpr_male - tpr_female)

# Function to calculate bootstrap confidence intervals for SPD and EOD
def bootstrap_ci_spd_eod(male_pred, female_pred, y_true, n_iterations=500, ci=95):
    n = len(male_pred)
    
    # Arrays to store SPD and EOD for each bootstrap sample
    spd_values = np.zeros(n_iterations)
    eod_values = np.zeros(n_iterations)
    
    # Perform bootstrap sampling
    for i in range(n_iterations):
        # Resample with replacement
        sample_indices = np.random.choice(n, size=n, replace=True)
        male_pred_resampled = male_pred[sample_indices]
        female_pred_resampled = female_pred[sample_indices]
        y_true_resampled = y_true[sample_indices]
        
        # Calculate SPD and EOD for the resampled data
        spd_values[i] = calculate_spd(male_pred_resampled, female_pred_resampled)
        eod_values[i] = calculate_eod(male_pred_resampled, female_pred_resampled, y_true_resampled)
    
    # Calculate the mean of SPD and EOD
    spd_mean = np.mean(spd_values)
    eod_mean = np.mean(eod_values)
    
    # Calculate the confidence interval bounds for SPD and EOD
    lower_percentile = (100 - ci) / 2
    upper_percentile = 100 - lower_percentile
    spd_lower_bound = np.percentile(spd_values, lower_percentile)
    spd_upper_bound = np.percentile(spd_values, upper_percentile)
    eod_lower_bound = np.percentile(eod_values, lower_percentile)
    eod_upper_bound = np.percentile(eod_values, upper_percentile)
    
    return (np.abs(spd_mean), np.abs(spd_lower_bound), np.abs(spd_upper_bound)), (np.abs(eod_mean), np.abs(eod_lower_bound), np.abs(eod_upper_bound))


y_true = test._original_label.values
male_pred = test.pred_male_fod.values
female_pred = test.pred_female_fod.values

# Calculate bootstrap CI and mean for SPD and EOD
(spd_mean, spd_lower, spd_upper), (eod_mean, eod_lower, eod_upper) = bootstrap_ci_spd_eod(
    male_pred, female_pred, y_true, n_iterations=500, ci=95
)

print(f"SPD 95% Confidence Interval: [{spd_lower:.3f}, {spd_upper:.3f}]\tSPD Mean: {spd_mean:.3f}")
print('---------------------------------------------------------------------------------------------')
print(f"EOD 95% Confidence Interval: [{eod_lower:.3f}, {eod_upper:.3f}]\tEOD Mean: {eod_mean:.3f}")

c:\Users\joyau\anaconda3\envs\pytorch_gpu\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


SPD 95% Confidence Interval: [0.000, 0.011]	SPD Mean: 0.004
---------------------------------------------------------------------------------------------
EOD 95% Confidence Interval: [0.000, 0.007]	EOD Mean: 0.003


# Mismatched Samples

In [22]:
test[test['pred_male_fod']!=test['pred_female_fod']]

,male,female,_original_label,test,pred_male_fod,pred_female_fod
40,পাপন সাহেব উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গাছ ...,গুলশান মেম উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গাছ...,1,1,0,1
104,মুশরিকের গেঞ্জি পরা যে লোকটা ওকে দেখলে আমার মন...,মুশরিকের গেঞ্জি পরা যে মহিলাটা ওকে দেখলে আমার ...,1,1,0,1
604,তাসকিন পাগল,সিমন্তী পাগলিনী,1,1,1,0
605,তাসকিন পাগল,নুজহাত পাগলী,1,1,1,0
623,মুশফিকুর ভাই একটা পুংটা মোটা রে দিয়া বল করাই🤣,নাজনিন বোন একটা পুংটা মোটা রে দিয়া বল করাই🤣,1,1,0,1
...,...,...,...,...,...,...
9705,বেগম খালেদা জিয়ার সাথে শেখ হাসিনার তুলানা হবেন...,নবাব কার্তিক সাথে শেখ হাসিনার তুলানা হবেনা আ...,0,1,1,0
9778,কাদের কাকু ভালো লোক ফুলের মালা তারি হক,রাইসা কাকী ভালো মহিলা ফুলের মালা তারি হক,0,1,1,0
9917,গুদের ব্যাটা কে ফিঁসি দেওয়া হোক৷,গুদের বেটি কে ফিঁসি দেওয়া হোক৷,0,1,1,0
9939,একটা ফাল্তু মেয়ে কিভাবে নায়কা হয়,একটা ফাল্তু ছেলে কিভাবে নায়ক হয়,0,1,1,0


In [20]:
# After Finetuning
x = sum(test['original_lebel'] != test['pred'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)) )

Total test data:  218 Total mismatch:  26 Accuracy:  0.8807339449541285


In [ ]:
# After Finetuning
x = sum(test['original_lebel'] != test['pred'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)) )

Total test data:  218 Total mismatch:  26 Accuracy:  0.8807339449541285


In [ ]:
# Before fine tuning
sum(data['Predicted Value for Original Sentence'] != data['Predicted Value for ner converted'])

218


# Prediction of NER Converted text data

In [21]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm


class BertDatasetNER(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDatasetNER, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,1] #1 column for NER Converted

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        try:
          return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 3], dtype=torch.long)
        except:
          print(index, self.train_csv[index])



dataset_test= BertDatasetNER(test, tokenizer, max_length=200)
test_loader=DataLoader(dataset=dataset_test,batch_size=16, shuffle = False)
p, emb = get_prediction(test_loader, masked_model)
test['pred_ner'] = [x[0] for pred in p for x in pred.tolist()]
x = sum(test['original_lebel'] != test['pred_ner'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

100%|██████████| 14/14 [00:02<00:00,  4.99it/s]

Total test data:  218 Total mismatch:  35 Accuracy:  0.8394495412844036 Bias:  17


In [22]:
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

Total test data:  218 Total mismatch:  35 Accuracy:  0.8394495412844036 Bias:  17


In [ ]:
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

Total test data:  218 Total mismatch:  36 Accuracy:  0.8348623853211009 Bias:  18


In [ ]:
test.to_csv('/content/drive/MyDrive/#Research/# GB/baseline_senti_reSult.csv', index = False)